In [1]:
#!pip install transformers==4.2.2

In [2]:
!nvidia-smi

Mon May 30 12:58:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:59:00.0 Off |                    0 |
| N/A   45C    P0    75W / 350W |   4300MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
from sklearn.utils import shuffle

In [4]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

In [5]:
import optuna

In [6]:
train_df = pd.read_csv('../data/markup2train.csv')
test_df = pd.read_csv('../data/markup2test.csv')
train_df.head(3)

,Unnamed: 0,index,path,original,new,var
0,118,131,0024c525-9f48-4e21-8eec-d00aea56deaa,В целях продвижения нового канала продаж «Цифр...,Цель продвижения канала продаж «Цифровой офис...,4
1,1119,1184,00e3fe8f-114e-4b83-a15f-aa7af38f931a,в разрезе центров прибыли верхнего уровня:,по прибыли верхнего уровня:,1
2,1033,1123,00e19c69-1d75-4459-b4e3-8256a6fbdaf3,в срок до 16.12.2021 включительно сформировать...,до 16.12.2021 определить победителей Акции;,4


In [7]:
train_df.shape

(5845, 6)

In [8]:
train_df=train_df[train_df['new'].isna()==False].reset_index()
train_df.shape

(5841, 7)

In [9]:
class PairsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx < len(self.x['input_ids'])
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n # * 2

In [10]:
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple
from sklearn.model_selection import train_test_split
import gc
from tqdm.auto import tqdm, trange

2022-05-30 12:59:01.152279: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [11]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [12]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [13]:
USE_FOCAL_LOSS = False


ce = torch.nn.CrossEntropyLoss(reduction='none')

def focal_loss(model, batch):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    output = model(**batch)
    logits = output.logits
    labels = batch['labels']
    probs = (torch.nn.functional.one_hot(labels, logits.shape[2]) * torch.nn.functional.softmax(logits, dim=2)).sum(2)
    batch['labels'][batch['labels']==0] = -100
    ce_result = ce(logits.permute(0, 2, 1), labels)
    loss = (((1 - probs) ** 5) * ce_result).mean()
    return loss

In [14]:
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0

    for batch in test_dataloader:
        with torch.no_grad():
            if USE_FOCAL_LOSS:
                loss = focal_loss(model, batch)
            else:
                batch['labels'][batch['labels']==0] = -100
                loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss
            num += len(batch) * loss.item()
            den += len(batch)
    val_loss = num / den
    return val_loss

In [15]:
import random
from copy import deepcopy


def train_loop(
    model, train_dataloader, val_dataloader, 
    lr=3e-5,
    weight_decay=1e-2,
    max_epochs=7,
    gradient_accumulation_steps=1, 
    early_stop_round=2
):
    cleanup()
    print(f"LR = {lr}, weight_decay={weight_decay}")
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=weight_decay)

    ewm_loss = 0
    step = 0
    model.train()
    best_model = None
    best_eval = float('inf')
    es = 0
    for epoch in trange(max_epochs):
        tq = tqdm(train_dataloader)
        for i, batch in enumerate(tq):
            try:
                if USE_FOCAL_LOSS:
                    loss = focal_loss(model, batch)
                else:
                    batch['labels'][batch['labels']==0] = -100
                    loss = model(**{k: v.to(model.device) for k, v in batch.items()}).loss          
                
                loss.backward()
            except Exception as e:
                print('error on step', i, e)
                loss = None
                cleanup()
                continue
#             if i and i % gradient_accumulation_steps == 0:
#             for p in model.parameters():
#                 if random.random() < 0.5:
#                     p.grad = None
            optimizer.step()
#             for p in model.parameters():
#                 p.grad = None
            optimizer.zero_grad()
            step += 1

            ewm_loss = loss.item()
            tq.set_description(f'loss: {ewm_loss:4.4f}')

        model.eval()
        eval_loss = evaluate_model(model, val_dataloader)
        model.train()
        print(f'epoch {epoch}, step {i}/{step}: train loss: {ewm_loss:4.4f}  val loss: {eval_loss:4.4f}')
        if eval_loss < best_eval:
            es = 0
            best_eval = eval_loss
            # Потом надо раскомментировать
            best_model = deepcopy(model)
            print("New best pretrain_iteration")
        else:
            es += 1
        if es == early_stop_round:
            print("Early Stop!")
            break
        cleanup()
    cleanup()
    return best_eval, best_model

In [16]:
def train_model(x, y, model_name, test_size=0.1, batch_size=32, **kwargs):
    model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    x1, x2, y1, y2 = train_test_split(x, y, test_size=test_size, random_state=42)
    train_dataset = PairsDataset(tokenizer(x1), tokenizer(y1))
    test_dataset = PairsDataset(tokenizer(x2), tokenizer(y2))
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator, pin_memory=True)
    val_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator, pin_memory=True)

    eval_loss, best_model = train_loop(model, train_dataloader, val_dataloader, **kwargs)
    return eval_loss

In [17]:
cleanup()

In [18]:
datasets = {
    'train': train_df[['original', 'new']],
    'test': test_df[['original', 'new']]
    
}

In [19]:
model_name = 'sberbank-ai/ruT5-large'

In [20]:
import optuna

In [21]:
def objective(trial):
    print(f'Trial {trial.number} is started')
    lr = trial.suggest_float("lr", 1e-6, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-7, 1e-1, log=True)
    seed = 0
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    d = datasets['train']
    dname = 'train'
    print(f'\n\n\n  train \n=====================\n\n')
    eval_loss = train_model(d['original'].tolist(),
                            d['new'].tolist(),
                            model_name=model_name,
                            batch_size=16,
                            lr=lr,
                            weight_decay=weight_decay,
                            )
    return eval_loss

In [22]:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)

In [92]:
study.best_params

{'lr': 6.170526429900845e-05, 'weight_decay': 3.150705016935834e-05}

In [93]:
study.best_trial

FrozenTrial(number=27, values=[0.33263646911930395], datetime_start=datetime.datetime(2022, 5, 27, 7, 17, 7, 520869), datetime_complete=datetime.datetime(2022, 5, 27, 7, 31, 7, 993769), params={'lr': 6.170526429900845e-05, 'weight_decay': 3.150705016935834e-05}, distributions={'lr': LogUniformDistribution(high=0.01, low=1e-06), 'weight_decay': LogUniformDistribution(high=0.1, low=1e-07)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=27, state=TrialState.COMPLETE, value=None)

In [97]:
# import pickle

# with open('optuna_history_t5_full_dataset.pkl', 'wb') as f:
#     pickle.dump(study, f)

In [96]:
import pickle

with open('optuna_history_t5_full_dataset.pkl', 'rb') as f:
    study = pickle.load(f)

In [98]:
def get_best_model(x, y, model_name, test_size=0.1, batch_size=32, **kwargs):
    model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    seed = 0
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    x1, x2, y1, y2 = train_test_split(x, y, test_size=test_size, random_state=42)
    train_dataset = PairsDataset(tokenizer(x1), tokenizer(y1))
    test_dataset = PairsDataset(tokenizer(x2), tokenizer(y2))
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator, pin_memory=True)
    val_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=True, collate_fn=data_collator, pin_memory=True)

    eval_loss, best_model = train_loop(model, train_dataloader, val_dataloader, **kwargs)
    return best_model

In [99]:
d = datasets['train']
model = get_best_model(d['original'].tolist(),
                        d['new'].tolist(),
                        model_name=model_name,
                        batch_size=16,
                        lr=study.best_params['lr'],
                        weight_decay=study.best_params['weight_decay'],
                        max_epochs=7
                        )

LR = 6.170526429900845e-05, weight_decay=3.150705016935834e-05


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/329 [00:00<?, ?it/s]

error on step 143 CUDA out of memory. Tried to allocate 702.00 MiB (GPU 0; 31.75 GiB total capacity; 25.34 GiB already allocated; 461.25 MiB free; 28.93 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 166 CUDA out of memory. Tried to allocate 720.00 MiB (GPU 0; 31.75 GiB total capacity; 25.26 GiB already allocated; 423.25 MiB free; 28.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 214 CUDA out of memory. Tried to allocate 670.00 MiB (GPU 0; 31.75 GiB total capacity; 25.27 GiB already allocated; 543.25 MiB free; 28.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See document

  0%|          | 0/329 [00:00<?, ?it/s]

...

error on step 12 CUDA out of memory. Tried to allocate 720.00 MiB (GPU 0; 31.75 GiB total capacity; 26.81 GiB already allocated; 181.25 MiB free; 29.20 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 58 CUDA out of memory. Tried to allocate 702.00 MiB (GPU 0; 31.75 GiB total capacity; 26.18 GiB already allocated; 271.25 MiB free; 29.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
epoch 4, step 328/1631: train loss: 0.2165  val loss: 0.0870
New best pretrain_iteration


  0%|          | 0/329 [00:00<?, ?it/s]

error on step 169 CUDA out of memory. Tried to allocate 720.00 MiB (GPU 0; 31.75 GiB total capacity; 26.09 GiB already allocated; 567.25 MiB free; 28.82 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 188 CUDA out of memory. Tried to allocate 702.00 MiB (GPU 0; 31.75 GiB total capacity; 26.16 GiB already allocated; 625.25 MiB free; 28.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
epoch 5, step 328/1958: train loss: 0.0639  val loss: 0.0807
New best pretrain_iteration


  0%|          | 0/329 [00:00<?, ?it/s]

error on step 106 CUDA out of memory. Tried to allocate 1.41 GiB (GPU 0; 31.75 GiB total capacity; 24.69 GiB already allocated; 1.06 GiB free; 28.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 221 CUDA out of memory. Tried to allocate 702.00 MiB (GPU 0; 31.75 GiB total capacity; 26.16 GiB already allocated; 357.25 MiB free; 29.03 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
error on step 234 CUDA out of memory. Tried to allocate 666.00 MiB (GPU 0; 31.75 GiB total capacity; 26.00 GiB already allocated; 497.25 MiB free; 28.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentatio

## Чекпойнты

In [102]:
# torch.save(model,'../models/t5_optuna_chekpoint_full_dataset.pt')

# torch.save(model,'../models/t5_focal_full_dataset.pt')

## Результаты

In [76]:
base_model_name = 'sberbank-ai/ruT5-large'
model_name = 't5_large_train_300'

In [77]:
print()

In [78]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

In [79]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3, repetition_penalty=1.0):
    model.eval()
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt')['input_ids'].to(model.device)

    if max_length == 'auto':
        max_length = inputs.shape[1]
    result = model.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, 
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
#         early_stopping=True,
#         top_k=50,
#         top_p=0.95,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [31]:
import sys
sys.path.insert(1, '../evaluation/')
from metrics import count_metrics

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [35]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [36]:
test_text = ['Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам в соответствии с Приложением к настоящему Распоряжению.',
            'В случае подтверждения устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно осуществляет тиражирование ПО в своих подразделениях и филиалах.',
            'Контроль за исполнением настоящего Распоряжения оставляю за собой.',
            'Утвердить перечень автоматизированных систем и информационных ресурсов Банка, доступных к подключению Cотрудникам (Приложение 1).']

## Результаты T5

In [37]:
for test_string in test_text:
    print(f"ORIGINAL: {test_string}")
    print("REPHRASED: ", paraphrase([test_string], model, beams=10)[0])
    print("\n####", end='\n\n')

ORIGINAL: Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам в соответствии с Приложением к настоящему Распоряжению.
REPHRASED:  Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам по Приложению к этому Распоряжению.
####

ORIGINAL: В случае подтверждения устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно осуществляет тиражирование ПО в своих подразделениях и филиалах.
REPHRASED:  При подтверждении устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно тиражирует ПО в подразделениях и филиалах.
####

ORIGINAL: Контроль за исполнением настоящего Распоряжения оставляю за собой.
REPHRASED:  Исполнение этого Распоряжения проконтролирую лично.
####

ORIGINAL: Утвердить перече

## Результаты T5 + FOCAL LOSS

In [38]:
ans_set = ["С января 2021 года использовать список операций и коэффициенты пересчета продуктов в условные продукты (УП) для менеджеров по продажам по Приложению этого Распоряжения.",
 "Если подтверждается, что катастрофическая или серьезная ошибка устранена, заказчик самостоятельно тиражирует ПО в своих подразделениях и филиалах.",
 "Исполнение этого Распоряжения лично проконтролирую.",
 "Предоставить список систем и ресурсов Банка, доступных к подключению Cотрудникам (Приложение 1)."]

In [39]:
for test_string in test_text:
    print(f"ORIGINAL: {test_string}")
    print("REPHRASED: ", paraphrase([test_string], model, beams=10)[0])
    print("\n####", end='\n\n')

ORIGINAL: Начиная с премирования по итогам работы за январь 2021 г., руководствоваться перечнем операций и коэффициентами пересчета продуктов в условные продукты (далее - УП) для менеджеров по продажам в соответствии с Приложением к настоящему Распоряжению.
REPHRASED:  С января 2021 года использовать список операций и коэффициенты пересчета продуктов в условные продукты (УП) для менеджеров по продажам по Приложению этого Распоряжения.

####

ORIGINAL: В случае подтверждения устранения Катастрофической или Серьезной ошибки Заказчик самостоятельно осуществляет тиражирование ПО в своих подразделениях и филиалах.
REPHRASED:  Если подтверждается, что катастрофическая или серьезная ошибка устранена, заказчик самостоятельно тиражирует ПО в своих подразделениях и филиалах.

####

ORIGINAL: Контроль за исполнением настоящего Распоряжения оставляю за собой.
REPHRASED:  Исполнение этого Распоряжения лично проконтролирую.

####

ORIGINAL: Утвердить перечень автоматизированных систем и информационн